<a href="https://colab.research.google.com/github/Magouo/watki/blob/master/Lista2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Oct 16 12:30:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import numpy as np
import time
N = 1024
A_cpu = np.random.rand(N, N).astype(np.float32)
B_cpu = np.random.rand(N, N).astype(np.float32)
start = time.time()
C_cpu = np.dot(A_cpu, B_cpu)
end = time.time()
print(f"Mnozenie macierzy na cpu trwało {end - start: .5f} sekund.")

Mnozenie macierzy na cpu trwało  0.03328 sekund.


In [ ]:
import cupy as cp
import time
N = 1024
A_cpu = np.random.rand(N, N).astype(np.float32)
B_cpu = np.random.rand(N, N).astype(np.float32)
A_gpu = cp.array(A_cpu)
B_gpu = cp.array(B_cpu)
start = time.time()
C_gpu = np.matmul(A_gpu, B_gpu)
cp.cuda.Stream.null.synchronize()
end = time.time()
print(f"Mnozenie macierzy na gpu trwało {end - start: .5f} sekund.")

Mnozenie macierzy na gpu trwało  0.62898 sekund.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 13.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=ce6adaba69ea00fd900f83e67c1712980a6a314e420fff8b76717335aa3a5df4
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda


In [ ]:
kernel_code = """
  __global__ void matrixMul(float *A, float *B, float *C, int N) {


 int row = blockIdx.y * blockDim.y + threadIdx.y;
 int col = blockIdx.x * blockDim.x + threadIdx.x;
 float sum = 0.0;
 if(row < N && col < N) {
 for (int k = 0; k < N; k++) {
 sum += A[row * N + k] * B[k * N + col];
 }
 C[row * N + col] = sum;
 }
  }
  """

In [ ]:
  del A_gpu, B_gpu

NameError: name 'A_gpu' is not defined

In [ ]:
import numpy as np
import cupy as cp
import time
import matplotlib.pyplot as plt
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
listX=[]
ListY=[]
ListXgpu =[]
ListYgpu=[]
Listxcpu=[]
ListYcpu =[]
N = 100
for i in range (5):
  A_cpu = np.random.rand(N, N).astype(np.float32)
  B_cpu = np.random.rand(N, N).astype(np.float32)
  start = time.time()
  C_cpu = np.dot(A_cpu, B_cpu)
  end = time.time()
  Listxcpu.append(N)
  ListYcpu.append(end - start)

  A_gpu = cp.array(A_cpu)
  B_gpu = cp.array(B_cpu)
  start = time.time()
  C_gpu = np.matmul(A_gpu, B_gpu)
  cp.cuda.Stream.null.synchronize()
  end = time.time()
  ListXgpu.append(N)
  ListYgpu.append(end - start)
  del A_gpu, B_gpu, C_gpu,
  mod = SourceModule(kernel_code)
  matrixMul = mod.get_function("matrixMul")
  A_gpu = cp.array(A_cpu)
  B_gpu = cp.array(B_cpu)
  C_cpu = np.empty((N, N), np.float32)

  A_gpu = cuda.mem_alloc(A_cpu.nbytes)
  B_gpu = cuda.mem_alloc(B_cpu.nbytes)
  C_gpu = cuda.mem_alloc(C_cpu.nbytes)

  cuda.memcpy_htod(A_gpu, A_cpu)
  cuda.memcpy_htod(B_gpu, B_cpu)

  block_size = (32, 32, 1)
  grid_size = (int(N/32), int(N/32), 1)

  start = time.time()
  matrixMul(A_gpu, B_gpu, C_gpu, np.int32(N), block=block_size, grid=grid_size)
  cuda.Context.synchronize()
  end = time.time()
  cuda.memcpy_dtoh(C_cpu, C_gpu)

  del A_gpu, B_gpu, C_gpu, A_cpu, B_cpu, C_cpu
  listX.append(N)
  ListY.append(end-start)
  N = N * 2

x = listX
y = ListY
xgpu = ListXgpu
ygpu = ListYgpu
xcpu = Listxcpu
ycpu = ListYcpu
yy=[3,6,9,12,15]
kernel, cpu, gpu = plt.subplot()
kernel.plot(x,y)
cpu = plt.plot(xgpu,ygpu)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

CUBLASError: CUBLAS_STATUS_EXECUTION_FAILED